In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt


CSV_PATH = "full_df.csv"
IMAGE_DIR = r"C:\Users\Justin\Downloads\archive\ODIR-5K\ODIR-5K\Training Images"


df = pd.read_csv(CSV_PATH)
df['filepath'] = df['filename'].apply(lambda x: os.path.join(IMAGE_DIR, x))
df['target'] = df['target'].astype(str)


train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['target'], random_state=42)


IMG_SIZE = (512, 512)
BATCH_SIZE = 16


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='target',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='target',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


input_shape = IMG_SIZE + (3,)
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
num_classes = len(train_gen.class_indices)
output = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.5, patience=2)
]


history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=callbacks
)


for layer in base_model.layers[:-40]:
    layer.trainable = False
for layer in base_model.layers[-40:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=callbacks
)


loss, accuracy = model.evaluate(val_gen)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")


model.save("resnet50_fundus_partial_finetune.keras")


plt.plot(history1.history['accuracy'] + history2.history['accuracy'], label='Train Accuracy')
plt.plot(history1.history['val_accuracy'] + history2.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()


Found 5113 validated image filenames belonging to 8 classes.
Found 1279 validated image filenames belonging to 8 classes.


C:\Users\Justin\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


C:\Users\Justin\AppData\Roaming\Python\Python311\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_4502']
Received: inputs=Tensor(shape=(None, 512, 512, 3))
  warnings.warn(msg)


320/320 ━━━━━━━━━━━━━━━━━━━━ 915s 3s/step - accuracy: 0.3201 - loss: 2.3621 - val_accuracy: 0.4511 - val_loss: 1.5859 - learning_rate: 1.0000e-04
Epoch 2/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 919s 3s/step - accuracy: 0.3954 - loss: 1.7152 - val_accuracy: 0.4496 - val_loss: 1.6260 - learning_rate: 1.0000e-04
Epoch 3/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 902s 3s/step - accuracy: 0.4418 - loss: 1.6398 - val_accuracy: 0.4496 - val_loss: 1.6311 - learning_rate: 1.0000e-04
Epoch 4/10
147/320 ━━━━━━━━━━━━━━━━━━━━ 6:35 2s/step - accuracy: 0.4361 - loss: 1.6090